In [4]:
%%capture
!/Users/shashankagarwal/opt/anaconda3/envs/thellmbook/bin/pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
!/Users/shashankagarwal/opt/anaconda3/envs/thellmbook/bin/pip install llama-cpp-python==0.2.69

## Get Phi-3 Mini
Fetch the compressed Phi-3 model with half precision of FP16 compared to the original model with full precision of FP32.

In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

## Load Model

In [6]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=0,  # All the layers of the model run on the GPU
    max_tokens=500,   # limits the output length (how much text the model will generate in response).
    n_ctx=2048,       # determines the input capacity (how much context the model can handle).
    seed=42,
    verbose=False
)

## Basic Chain
Combines Prompt with LLM Model

In [7]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>{input_prompt}<|end|><|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

# Connect the prompt module with the model
basic_chain = prompt | llm

# Input the prompt
basic_chain.invoke(
    {
    "input_prompt": "How many states are there in Germany?"
    }
)

/Users/shashankagarwal/opt/anaconda3/envs/thellmbook/lib/python3.10/site-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  self._input_ids.tolist(), prompt_tokens


' As of my knowledge cutoff date in 2023, Germany is comprised of 16 federal states, known as Bundesländer. These states each have their own government and certain powers delegated to them by the German federal system. The states play a significant role in various areas such as education, law enforcement, and cultural affairs within their territories.'

## Multiple Chains
Combines multiple modules together

In [12]:
from langchain import LLMChain

product_template = """<s><|user|>Suggest one product in 2-5 words which could be advertised based on a short description: {idea}. Only return the product which could be advertised.<|end|><|assistant|>"""
product_prompt = PromptTemplate(
    template=product_template,
    input_variables=["idea"]
)
product_description = LLMChain(llm=llm, prompt=product_prompt, output_key="product_description")
product_description.invoke(
    {"idea": "An old lady is knitting a pullover while humming a tune"}
)

/Users/shashankagarwal/opt/anaconda3/envs/thellmbook/lib/python3.10/site-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  self._input_ids.tolist(), prompt_tokens


{'idea': 'An old lady is knitting a pullover while humming a tune',
 'product_description': ' Yarn for cozy sweaters.'}

In [13]:
product_name_slogan_template = """<s><|user|>Suggest one name and one slogan for the product based on the idea: {idea} and product: {product_description}<|end|><|assistant|>"""
product_name_slogan_prompt= PromptTemplate(
    template=product_name_slogan_template,
    input_variables=["idea", "product_description"]
)
product_name_slogan = LLMChain(llm=llm, prompt=product_name_slogan_prompt, output_key="product_name_slogan")

In [14]:
product_ad_template = """<s><|user|>Give a really short advertisement idea to pitch to the team in 4-5 sentences based on the idea: {idea}, product description: {product_description}. Also, consider the name and slogan of the product, which is: {product_name_slogan}<|end|><|assistant|>"""
product_ad_prompt = PromptTemplate(
    template=product_ad_template,
    input_variables=["idea", "product_name_slogan", "product_description"]
)

product_ad = LLMChain(llm=llm, prompt=product_ad_prompt, output_key="product_ad")

In [15]:
product_chain = product_description | product_name_slogan | product_ad
product_chain.invoke("An old lady is knitting a pullover while humming a tune")

/Users/shashankagarwal/opt/anaconda3/envs/thellmbook/lib/python3.10/site-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  self._input_ids.tolist(), prompt_tokens
/Users/shashankagarwal/opt/anaconda3/envs/thellmbook/lib/python3.10/site-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  self._input_ids.tolist(), prompt_tokens
/Users/shashankagarwal/opt/anaconda3/envs/thellmbook/lib/python3.10/site-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  self._input_ids.tolist(), prompt_tokens


{'idea': 'An old lady is knitting a pullover while humming a tune',
 'product_description': ' Yarn or Knitting Needles Set!',
 'product_name_slogan': ' Name: "Knit-Nurture Essentials"\n\nSlogan: "Stitch by Stitch, Crafting Comfort for Every Season!"',
 'product_ad': ' Introducing "Knit-Nurture Essentials," a Yarn or Knitting Needles Set designed with the crafter in mind. Picture our beloved grandmother, knitting cozily while humming along to her favorite tunes—now imagine bringing that comfort and warmth into every room of your home. With "Stitch by Stitch, Crafting Comfort for Every Season!" at hand, create a personalized pullover just like our inspiration. Join the Knit-Nurture family today and knit your way to a world full of homemade charm!'}